<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20A2D%20Action%20Recognition%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents

## Installation

## Training your own classifier

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/18_mmaction/installation
     
 - Select the right file and run
 
     - chmod +x install.sh && ./install.sh

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
! cd Monk_Object_Detection/18_mmaction/installation && chmod +x install.sh && ./install.sh

# Download and reformat Dataset
  - Dataset Credits: http://web.eecs.umich.edu/~jjcorso/r/a2d/

In [ ]:
! wget http://web.eecs.umich.edu/~jjcorso/bigshare/A2D_main_1_0.tar.bz

In [ ]:
! bzip2 -d A2D_main_1_0.tar.bz

In [ ]:
! tar -xf A2D_main_1_0.tar

In [ ]:
# copy the following lines to classes.txt

Background
adult-climbing
adult-crawling
adult-eating
adult-flying
adult-jumping
adult-rolling
adult-running
adult-walking
adult-none
baby-climbing
baby-crawling
baby-eating
baby-flying
baby-jumping
baby-rolling
baby-running
baby-walking
baby-none
ball-climbing
ball-crawling
ball-eating
ball-flying
ball-jumping
ball-rolling
ball-running
ball-walking
ball-none
bird-climbing
bird-crawling
bird-eating
bird-flying
bird-jumping
bird-rolling
bird-running
bird-walking
bird-none
car-climbing
car-crawling
car-eating
car-flying
car-jumping
car-rolling
car-running
car-walking
car-none
cat-climbing
cat-crawling
cat-eating
cat-flying
cat-jumping
cat-rolling
cat-running
cat-walking
cat-none
dog-climbing
dog-crawling
dog-eating
dog-flying
dog-jumping
dog-rolling
dog-running
dog-walking
dog-none

In [ ]:
# changing ID numbers, making them continuous for training

In [ ]:
! mkdir videos

In [ ]:
import os
import pandas as pd
import numpy as np

f = open("Release/videoset.csv");
lines = f.readlines();
f.close()

combined = [];
from tqdm import tqdm

for i in tqdm(range(len(lines))):
    tmp = lines[i].split(",");
    in_vid = "Release/clips320H/" + tmp[0] + ".mp4";
    class_id = int(tmp[1])-10;
    st = tmp[2];
    et = tmp[3];
    out_vid = "videos/" + str(i) + ".mp4"
    
    cmd = "ffmpeg -ss " + st + " -i " + in_vid + " -to " + et + " -c copy " + out_vid
    
    os.system(cmd)
    
    wr = out_vid + " " + str(class_id);
    combined.append(wr)
    
    #print(cmd)
    #print(in_vid, os.path.isfile(in_vid))
    #break;

In [ ]:
import random
random.shuffle(combined)

In [ ]:
train_list = combined[:int(len(combined)*0.8)];
val_list = combined[int(len(combined)*0.8):];

In [ ]:
f = open("train.txt", 'w')
for i in range(len(train_list)):
    f.write(train_list[i] + "\n");
f.close();

In [ ]:
f = open("val.txt", 'w')
for i in range(len(val_list)):
    f.write(val_list[i] + "\n");
f.close();

# Train your own network

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/18_mmaction/lib");

In [ ]:
from train_engine import Detector_Videos

In [ ]:
gtf = Detector_Videos();

In [ ]:
video_dir = './';
anno_file = 'train.txt';
classes_list_file = 'classes.txt';

gtf.Train_Video_Dataset(video_dir, 
                          anno_file, 
                          classes_list_file);

In [ ]:
video_dir = './';
anno_file = 'val.txt';

gtf.Val_Video_Dataset(video_dir, 
                          anno_file);

In [ ]:
gtf.Dataset_Params(batch_size=4, num_workers=4)

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model_name="tsm_r50", gpu_devices=[0])

In [ ]:
gtf.Hyper_Params(lr=0.002, momentum=0.9, weight_decay=0.0001)

In [ ]:
gtf.Training_Params(num_epochs=100, val_interval=10)

In [ ]:
gtf.Train();

# Inference on trained network

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/18_mmaction/lib");

In [ ]:
from infer_engine import Infer_Videos

In [ ]:
gtf = Infer_Videos();

In [ ]:
gtf.Dataset_Params('classes.txt');

In [ ]:
config_file = "work_dirs/config.py"
checkpoint_file = "work_dirs/latest.pth"

gtf.Model_Params(config_file, checkpoint_file, use_gpu=True)

In [ ]:
import os
vid_list = os.listdir("videos")

In [ ]:
video_path = "videos/" + vid_list[0]
results = gtf.Predict(video_path)
from IPython.display import Video
Video(video_path)

In [ ]:
video_path = "videos/" + vid_list[1]
results = gtf.Predict(video_path)
from IPython.display import Video
Video(video_path)